In [369]:
#Install Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [374]:
#Upload Data
datafile = "/Users/justinpassek/Downloads/481802_SoFi_Creative_Fatigue_Testing_20191211_100848_2743124657.csv"
DF = pd.read_csv(datafile
               ,skiprows=10
                )
print(DF)

                              Campaign                  Creative        Date  \
0       A_BR_GOOG_YouTube-Anthem_Y19Q3  BR_Anthem-15_TRACK_1x1_2  2019-09-16   
1       A_BR_GOOG_YouTube-Anthem_Y19Q3  BR_Anthem-15_TRACK_1x1_2  2019-09-16   
2       A_BR_GOOG_YouTube-Anthem_Y19Q3  BR_Anthem-15_TRACK_1x1_2  2019-09-16   
3       A_BR_GOOG_YouTube-Anthem_Y19Q3  BR_Anthem-15_TRACK_1x1_2  2019-09-16   
4       A_BR_GOOG_YouTube-Anthem_Y19Q3  BR_Anthem-15_TRACK_1x1_2  2019-09-16   
...                                ...                       ...         ...   
109624                    SoFi H1 2017         Tracking Creative  2019-12-06   
109625                    SoFi H1 2017         Tracking Creative  2019-12-10   
109626             Turbo Affiliate Tag             2967 Tracking  2019-09-30   
109627             Turbo Affiliate Tag             2968 Tracking  2019-09-30   
109628                    Grand Total:                       ---         ---   

       Creative ID  Impressions   Click

In [375]:
#Preprocess Data
#df with Important Columns: 'Creative_id' , 'Date' , 'General_bucket' , 'CTR'
#A) Change Column Names. old_name:new_name
new_names = {'Campaign':'Campaign',
            'Creative ID':'Creative_id',
            'Date':'Date',
#            'old_name':'General_bucket',
            'Impressions':'Impressions',
            'Clicks':'Clicks'
            }
DF.rename(columns=new_names,inplace=True)

#B) Clean out 'Creative_id' without Adequate Metrics
#20 Impression Minimum 
DF = DF[DF["Impressions"]>1000]

#1 Click minimum
DF = DF[DF["Clicks"]>1]

#'Impressions' Must be Greater than the Clicks
DF = DF[DF["Impressions"]>DF["Clicks"]]

#Remove Unidentifiable Values/Rows
DF = DF[DF.Creative_id != '(not set)']
DF = DF[DF.Creative_id != '---']

#C) Add 'CTR' Column
DF['CTR']=DF.Clicks / DF.Impressions

DF['Date'] = pd.to_datetime(DF['Date'])

print(DF)

                                          Campaign                  Creative  \
0                   A_BR_GOOG_YouTube-Anthem_Y19Q3  BR_Anthem-15_TRACK_1x1_2   
1                   A_BR_GOOG_YouTube-Anthem_Y19Q3  BR_Anthem-15_TRACK_1x1_2   
2                   A_BR_GOOG_YouTube-Anthem_Y19Q3  BR_Anthem-15_TRACK_1x1_2   
3                   A_BR_GOOG_YouTube-Anthem_Y19Q3  BR_Anthem-15_TRACK_1x1_2   
4                   A_BR_GOOG_YouTube-Anthem_Y19Q3  BR_Anthem-15_TRACK_1x1_2   
...                                            ...                       ...   
109483  SoFi - Paid Social - Reddit - Q3FY19 - SLR                        H9   
109484  SoFi - Paid Social - Reddit - Q3FY19 - SLR                        H9   
109485  SoFi - Paid Social - Reddit - Q3FY19 - SLR                        H9   
109486  SoFi - Paid Social - Reddit - Q3FY19 - SLR                        H9   
109487  SoFi - Paid Social - Reddit - Q3FY19 - SLR                        H9   

             Date Creative_id  Impressi

In [372]:
#Filter by 'General_bucket' (IF NECESSARY)
df = DF
#df = DF[DF['Campaign'].str.contains("_MTG_")]
print(df)

In [373]:
#Table_1 Granularity: 'Creative_id' level
imp_columns_1 = ['Campaign',
               "Creative_id",
               "Date",
#               "General_bucket",
               "CTR",
               "Impressions",
               "Clicks"
                ]
Table_1 = df[imp_columns_1]

#print(Table_1)

#Table_2 with Important Columns: 'Date' , 'Impressions' , 'Clicks' , 'CTR'
imp_columns_2 = ["Date",
                 "Impressions",
                 "Clicks"
                ]
Table_2 = df[imp_columns_2]

#Table_2 Granularity: Date level
Table_2 = Table_2.groupby(['Date']).sum()

#Reset (remove) Date from Index to Column
Table_2.reset_index(level=0, inplace=True)

#Add 'CTR' Column
Table_2['CTR'] = Table_2.Clicks / Table_2.Impressions

#print(Table_2)

In [334]:
#Table_1 Add Column: '7_day_MA'
Table_1['7_day_MA'] = Table_1.CTR.rolling(window=7).mean()

#Table_2 Add Column: '90_day_MA'
Table_2['90_day_MA'] = Table_2.CTR.rolling(window=90).mean()

# DO PROSPECTING VS REMARKETING

#print(Table_1)
#print(Table_2)

In [366]:
#Join '90_day_MA' to Table_1 on 'Date'
Table_2 = Table_2[['Date',
                  '90_day_MA'
                  ]]

#Table_2 = Table_2.loc[:, ['Date','90_day_MA']]
print(Table_2)
Table_3 = Table_1.join(Table_2.set_index('Date'), on='Date')

#print(Table_3)

          Date  90_day_MA
0   2018-12-11        NaN
1   2018-12-12        NaN
2   2018-12-13        NaN
3   2018-12-14        NaN
4   2018-12-15        NaN
..         ...        ...
360 2019-12-06   0.001121
361 2019-12-07   0.001087
362 2019-12-08   0.001103
363 2019-12-09   0.001097
364 2019-12-10   0.001090

[365 rows x 2 columns]


In [336]:
# #Re-name/store Table_3 as a Variable Correlating with 'General Bucket'
# Table_3['Product'] = 'MTG'
# MTG = Table_3
# #Re-run through this Step as Many Times as Required
# #(NOTE: Build a Function after POC)

In [337]:
# #Calculate Intersection_Table
# #Re-Aggregate Tables together
# Int_Table_1 = pd.concat([Money
#                        ,Brokerage
#                        ,Crypto
#                          ,Invest
#                          ,PL
#                          ,PSL
#                          ,SLR
#                          ,MTG
#                         ], ignore_index=True)
# print(Int_Table_1)

In [338]:
#Add Column to Int_Table_1 Calculating "Lift" by 'Creative_id' and 'Date'
Int_Table_1 = Table_3
Int_Table_1['MA_Diff_7-90'] = Int_Table_1['7_day_MA'] - Int_Table_1['90_day_MA']
print(Int_Table_1)

#Derive column "Max_Lift" in Int_Table_2
#Int_Table_2 = Int_Table_1
Int_Table_2 = Int_Table_1.groupby('Creative_id', as_index=False)['MA_Diff_7-90'].max()
Int_Table_2.rename(columns={"MA_Diff_7-90":"Max_Lift"},inplace=True)
Int_Table_2 = Int_Table_1.join(Int_Table_2.set_index('Creative_id'), on='Creative_id')
#print(Int_Table_2)

#Derive column "Max_Date"
Int_Table_3 = Int_Table_2.loc[Int_Table_2['MA_Diff_7-90'] == Int_Table_2['Max_Lift']]
Int_Table_3.rename(columns={"Date":"Max_Date"},inplace=True)
#print(Int_Table_3)

#Join 'Max_Date'
Int_Table_4 = Int_Table_1.join(Int_Table_3[['Creative_id','Max_Date']].set_index('Creative_id'), on='Creative_id')
#print(Int_Table_4)

                                          Campaign Creative_id       Date  \
4                   A_BR_GOOG_YouTube-Anthem_Y19Q3   121528767 2019-09-16   
5                   A_BR_GOOG_YouTube-Anthem_Y19Q3   121529160 2019-09-16   
6                   A_BR_GOOG_YouTube-Anthem_Y19Q3   121530731 2019-09-16   
7                   A_BR_GOOG_YouTube-Anthem_Y19Q3   121568707 2019-09-16   
8                   A_BR_GOOG_YouTube-Anthem_Y19Q3   121568710 2019-09-16   
...                                            ...         ...        ...   
171082  SoFi - Paid Social - Reddit - Q3FY19 - SLR   119045579 2019-09-26   
171083  SoFi - Paid Social - Reddit - Q3FY19 - SLR   119045579 2019-09-27   
171084  SoFi - Paid Social - Reddit - Q3FY19 - SLR   119045579 2019-09-28   
171085  SoFi - Paid Social - Reddit - Q3FY19 - SLR   119045579 2019-09-29   
171086  SoFi - Paid Social - Reddit - Q3FY19 - SLR   119045579 2019-09-30   

             CTR  Impressions  Clicks  7_day_MA  90_day_MA  MA_Diff_7-90  


/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [339]:
#Filter for rows with 'Date' After 'Max_Date'
Int_Table_5 = Int_Table_4[Int_Table_4.Date > Int_Table_4.Max_Date]
#print(Int_Table_5)

#Filter for Rows with Negative Values
Int_Table_5 = Int_Table_5[Int_Table_5['MA_Diff_7-90'] <.0000000001]
#print(Int_Table_5)

#Find the Earliest 'Date'
Int_Table_5 = Int_Table_5.groupby('Creative_id').agg({'Date':np.min})

#Reset (remove) 'Creative_id' from Index to Column
Int_Table_5.reset_index(level=0, inplace=True)
Int_Table_5.rename(columns={"Date":"Intersection_Date"},inplace=True)
#print(Int_Table_5)

#Join Intersection Date to Creative level table
Int_Table_6 = Int_Table_5.join(Int_Table_3.set_index('Creative_id'), on='Creative_id')
#print(Int_Table_6)

In [365]:
#Derive column Decay
Int_Table_6['Decay'] = Int_Table_6['Intersection_Date'] - Int_Table_6['Max_Date']
Int_Table_6['Decay'] = Int_Table_6['Decay']/np.timedelta64(1,'D')
Final_Intersection_Table = Int_Table_6[['Creative_id',
#                                        'Product',
                                      'Campaign',
                                      'Max_Lift',
                                       'Max_Date',
                                      'Intersection_Date',
                                      'Decay']]
print(Final_Intersection_Table)

    Creative_id                                      Campaign  Max_Lift  \
0     108478486  R_Brokerage_DV360_Twitter_March_Madness_2019 -0.000035   
1     108478510  R_Brokerage_DV360_Twitter_March_Madness_2019 -0.000267   
2     108532031  R_Brokerage_DV360_Twitter_March_Madness_2019 -0.000288   
3     108532322  R_Brokerage_DV360_Twitter_March_Madness_2019 -0.000438   
4     108533795  R_Brokerage_DV360_Twitter_March_Madness_2019 -0.000144   
..          ...                                           ...       ...   
714   125233728                   P_PSL_DV360_EVERGREEN_Y19Q2 -0.000310   
715   125234745                   P_PSL_DV360_EVERGREEN_Y19Q2 -0.000397   
716   125349835                   P_PSL_DV360_EVERGREEN_Y19Q2 -0.000590   
717   125349859                   P_PSL_DV360_EVERGREEN_Y19Q2 -0.000792   
718   125349865                   P_PSL_DV360_EVERGREEN_Y19Q2 -0.000635   

      Max_Date Intersection_Date  Decay  
0   2019-03-23        2019-03-24    1.0  
1   2019-03-23 

In [341]:
# #Add Product Column
# def f(row):
#     if Final_Intersection_Table[Final_Intersection_Table['Campaign'].str.startswith("P_MTG_")]:
#         val = 'P_MTG'
#     elif Final_Intersection_Table[Final_Intersection_Table['Campaign'].str.startswith("P_Money_")]:
#         val = 'P_Money'
#     elif Final_Intersection_Table[Final_Intersection_Table['Campaign'].str.startswith("P_Brokerage_")]:
#         val = 'P_Brokerage'    
#     elif Final_Intersection_Table[Final_Intersection_Table['Campaign'].str.startswith("P_Invest_")]:
#         val = 'P_Invest'
#     elif Final_Intersection_Table[Final_Intersection_Table['Campaign'].str.startswith("P_Crypto_")]:
#         val = 'P_Crypto'     
#     elif Final_Intersection_Table[Final_Intersection_Table['Campaign'].str.startswith("P_PL_")]:
#         val = 'P_PL'
#     elif Final_Intersection_Table[Final_Intersection_Table['Campaign'].str.startswith("P_PSL_")]:
#         val = 'P_PSL'
#     elif Final_Intersection_Table[Final_Intersection_Table['Campaign'].str.startswith("P_SLR_")]:
#         val = 'P_SLR'
#     elif Final_Intersection_Table[Final_Intersection_Table['Campaign'].str.startswith("_MTG_")]:
#         val = 'R_MTG'
#     elif Final_Intersection_Table[Final_Intersection_Table['Campaign'].str.startswith("R_Money_")]:
#         val = 'R_Money'
#     elif Final_Intersection_Table[Final_Intersection_Table['Campaign'].str.startswith("R_Brokerage_")]:
#         val = 'R_Brokerage'    
#     elif Final_Intersection_Table[Final_Intersection_Table['Campaign'].str.startswith("R_Invest_")]:
#         val = 'R_Invest'
#     elif Final_Intersection_Table[Final_Intersection_Table['Campaign'].str.startswith("R_Crypto_")]:
#         val = 'R_Crypto'     
#     elif Final_Intersection_Table[Final_Intersection_Table['Campaign'].str.startswith("R_PL_")]:
#         val = 'R_PL'
#     elif Final_Intersection_Table[Final_Intersection_Table['Campaign'].str.startswith("R_PSL_")]:
#         val = 'R_PSL'
#     elif Final_Intersection_Table[Final_Intersection_Table['Campaign'].str.startswith("R_SLR_")]:
#         val = 'R_SLR'
#     elif Final_Intersection_Table[Final_Intersection_Table['Campaign'].str.startswith("C_Brokerage_")]:
#         val = 'C_Brokerage'           
#     else:
#         val = 'NA'
#     return val

# Final_Intersection_Table['Product'] = Final_Intersection_Table.apply(f,axis=1)

In [346]:
#Text-to-Columns
Delimited_Campaign = Final_Intersection_Table['Campaign'].str.split('_', n=5, expand=True)
Delimited_Campaign.columns = ['Delimited_id{}'.format(x+1) for x in Delimited_Campaign.columns]

#Table with only the Relevant Columns
Delimited_Campaign = Delimited_Campaign[['Delimited_id1',
                                        'Delimited_id2',
                                        'Delimited_id4']]

#Rename Columns
Delimited_Campaign.rename(columns={"Delimited_id1":"Campaign_Type",
                                  "Delimited_id2":"Product_Line",
                                  "Delimited_id4":"Evergreen"},inplace=True)

print(Delimited_Campaign)
print(Final_Intersection_Table)

#Join Relevant Delimited Columns back to Table
Final_Intersection_Table = Final_Intersection_Table.join(Delimited_Campaign)

print(Final_Intersection_Table)

    Campaign_Type Product_Line  Evergreen
0               R    Brokerage    Twitter
1               R    Brokerage    Twitter
2               R    Brokerage    Twitter
3               R    Brokerage    Twitter
4               R    Brokerage    Twitter
..            ...          ...        ...
714             P          PSL  EVERGREEN
715             P          PSL  EVERGREEN
716             P          PSL  EVERGREEN
717             P          PSL  EVERGREEN
718             P          PSL  EVERGREEN

[719 rows x 3 columns]
    Creative_id                                      Campaign  Max_Lift  \
0     108478486  R_Brokerage_DV360_Twitter_March_Madness_2019 -0.000035   
1     108478510  R_Brokerage_DV360_Twitter_March_Madness_2019 -0.000267   
2     108532031  R_Brokerage_DV360_Twitter_March_Madness_2019 -0.000288   
3     108532322  R_Brokerage_DV360_Twitter_March_Madness_2019 -0.000438   
4     108533795  R_Brokerage_DV360_Twitter_March_Madness_2019 -0.000144   
..          ...       

In [362]:
#Convert String to Title-Style to Encompass all Instances of Evergreen
Final_Intersection_Table['Evergreen'] = Final_Intersection_Table['Evergreen'].str.title()
Final_Intersection_Table = Final_Intersection_Table[Final_Intersection_Table.Evergreen == 'Evergreen']

In [364]:
#Pivot Tables
#PV Function
def pivot_tbl_2_dim(x,y):
    pvdf = df.groupby([x,y],as_index=False)[('Max_Lift')].sum()
#    pvdf['CTR'] = pvdf.Clicks/pvdf.Impressions
#    pvdf['SPV_Rate'] = pvdf.SPV/pvdf.Impressions
    return pvdf
print(pivot_tbl_2_dim(Final_Intersection_Table.Campaign_Type,Final_Intersection_Table.Product_Line))

KeyError: 'Column not found: Max_Lift'

In [320]:
#Write file
Final_Intersection_Table.to_csv("/Users/justinpassek/Downloads/Final_Intersection_Table3.csv", index=False)